In [44]:
import os
from dotenv import load_dotenv
from datetime import date, timedelta, datetime

from google.cloud import aiplatform, bigquery
from langchain_google_vertexai import VertexAI
import pandas as pd
from pandas import DataFrame

In [45]:
from crewai import Crew, Agent, Task, Process
from langchain.tools import tool

In [ ]:
load_dotenv('.env')
client = os.getenv('CLIENT_DATASET_NAME')
project_id = os.getenv('PROJECT_ID')
print(client, project_id)

In [ ]:
"""Initialize llm"""
aiplatform.init()
llm_0=VertexAI(model_name='gemini-2.5-flash-001', candidate_count=1, temperature=0.4, max_tokens= 1024)
llm_1=VertexAI(model_name='gemini-2.5-flash-001', candidate_count=1, temperature=0.4, max_tokens= 1024)
llm_2=VertexAI(model_name='gemini-2.5-flash-001', candidate_count=1, temperature=0.4, max_tokens= 1024)
# llm=ChatVertexAI(model_name='chat-bison@001')
# llm = ChatVertexAI(model_name='codechat-bison@001')
print(llm_0)


In [48]:
import json

def responseParser(output:str, key:str='text'):
    """Parses the output. default output key is 'text' """
    output=output.replace('```json','')
    output=output.removeprefix('```json')
    output=output.replace('```','')
    output = output.lstrip()
    # output = json.loads(output)
    # print(output)
    # return output[key]
    return output

def saveInsights(task_output):
    current_datetime = datetime.now()
    formatted_datetime = current_datetime.strftime('%Y%m%d-%H%M')
    filename = f"{formatted_datetime}.json"
    task_output = str(task_output)
    task_output = responseParser(task_output)
    # print(task_output)
    with open(filename,'w') as file:
        file.write(str(task_output))
    print(f"Result saved as {filename}")


In [ ]:
manager_agent = Agent(
    role="Manager",
    goal="""Provide final answer to user {Question} from input. To do so, you may need to delegate the task and efficiently manage the crew and ensure high-quality task completion.""",
    backstory="""
    Your role is to coordinate the efforts of the crew members, ensuring that only the required task is completed and to the highest standard, to answer the user {Question}.
    You are responsible for answering any question a user has asked. You will do so by managing the efficiently managing calls to the crew members. And report back the final answer to the user.
    Always introduce yourself as MANAGER in each response.
    """,
    tools=[],
    llm=llm_0,
    max_iter=5,
    allow_delegation=True,
    verbose=True,
    memory=True,
)

In [ ]:
assembly_summary = json.load(open("../data/assembly_summary.json"))

In [ ]:
provided_knowledge = f"""
        1) FlightScheduler - It's a great producitivity tool.
        2) assembly - It's a feature in FlightScheduler application.
"""

In [ ]:
FlightScheduler_assistant = Agent(
    role="FlightScheduler assistant",
    goal="""Answers any question about FlightScheduler application only with the provided_knowledge section.""", 
    backstory=(f"""
    You are responsible for answering any question about FlightScheduler application. Always introduce yourself as Frankie in each response.
    Very important that your knowledge about FlightScheduler is only limited to below and nothing else - 
        {provided_knowledge}
    You are not allowed to answer outside of this.
    """),
    tools=[],
    llm=llm_0,
    max_iter=5,
    allow_delegation=False,
    verbose=True,
    memory=True,
    )

In [60]:
context_builder = Agent(
    role="Accumulate data about user question",
    goal="""Your job is to build a detailed information set needed to answer a user question.""", 
    backstory=(f"""
    You have absolutely zero knowledge. You know nothing. To understand the user question you need to first gather information by asking relevant and basic questions.
    """),
    tools=[],
    llm=llm_1,
    allow_delegation=True,
    max_iter=5,
    verbose=True,
    memory=True,
    )

In [ ]:
insight_generator = Agent(
    role="insights generator about performing assembly",
    goal="""Your job is to generate insights in json format using the assembly_summary.""", 
    backstory=(f"""
    You have absolutely no knowledge of anything. First understand the context and build your knowledge by asking questions. Once you have sufficient knowledge, only then move to next step of building the insights - 
    assembly_summary below - 
    {assembly_summary}
    """),
    tools=[],
    llm=llm_2,
    allow_delegation=True,
    max_iter=5,
    verbose=True,
    memory=True,
    )

In [ ]:
UserQuery = f"""
        Can I have some insights about great assembly in FlightScheduler?
        """

In [ ]:
context_builder_task = Task(description=(f"""
    You have absolutely zero knowledge. You know nothing. To understand the user question you need to first gather information by asking relevant and basic questions to FlightScheduler assistant only.
    {UserQuery}
    """
    ),
    expected_output="""
    String format answer to question/input
    """,
    tools=[],
    agent=context_builder,
    )

In [64]:
requested_task = Task(description=(f"""
    {UserQuery}
    """
    ),
    expected_output="""
    String format answer to question/input
    """,
    tools=[],
    agent=manager_agent,
    context=[context_builder_task]
    )


In [ ]:
crew = Crew(
    # tasks=[ insight_generator_task, FlightScheduler_representative_task],
    # tasks=[requested_task, FlightScheduler_assistant_task, insight_generator_task],
    tasks=[context_builder_task],
    agents=[manager_agent, insight_generator, FlightScheduler_assistant, context_builder],
    # manager_llm = llm_0,
    # manager_agent = manager_agent,
    # process= Process.hierarchical,
    process= Process.sequential,
    )

# result= crew.kickoff(inputs={'Question':'What is FlightScheduler? Can I have some insights about great assembly?'})
result= crew.kickoff(inputs={})

print(result)